In [2]:
import geopandas as gpd
import pandas as pd
import tobler
import requests

https://figshare.com/articles/dataset/Data_for_Methodological_Foundation_of_a_Numerical_Taxonomy_of_Urban_Form_research_paper/16897102

In [2]:
url = "https://figshare.com/ndownloader/files/31252828"

In [3]:
r = requests.get(url)

In [5]:
with open("geoms.gpkg.zip", "wb") as f:
    f.write(r.content)

Unzip

In [3]:
buildings = gpd.read_file("prg_geometry.gpkg", engine='pyogrio', layer="buildings")

In [4]:
buildings = buildings.set_geometry(buildings.centroid)

In [5]:
clusters = pd.read_csv("https://figshare.com/ndownloader/files/31250203", index_col=0)

In [6]:
buildings = buildings.merge(clusters, on="uID")

In [7]:
buildings.cluster.nunique()

10

In [19]:
prague = gpd.read_file(
    "https://martinfleischmann.net/sds/chapter_04/data/zsj_prague_2021.gpkg"
)

In [9]:
buildings = buildings.sjoin(prague.to_crs(buildings.crs).drop(columns="n_people"), how='left')

In [56]:
r = requests.get("https://figshare.com/ndownloader/files/31250227")
with open("prg_primary.csv.gz", "wb") as f:
    f.write(r.content)

In [11]:
data = pd.read_csv("prg_primary.csv.gz", index_col=0)

In [14]:
data = buildings.merge(data, on="uID")

In [21]:
data[["geometry", "cluster_x", "sicFAR", "sdbHei", "ssbCCo", "stbSAl", "ltbIBD", "ldbPWL", "NAZ_ZSJ", "NAZ_KU"]].rename(columns={
    "cluster_x": "cluster",
    "sicFAR": "floor_area_ratio",
    "sdbHei": "height",
    "ssbCCo": "compactness",
    "stbSAl": "street_alignment",
    "ltbIBD": "interbuilding_distance",
    "ldbPWL": "block_perimeter_wall_length",
}).to_file("../../chapter_11/data/prg_building_locations.gpkg", engine="pyogrio")